## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carupano,VE,10.6678,-63.2585,77.83,89,96,8.93,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,73.45,68,75,9.22,broken clouds
2,2,Butaritari,KI,3.0707,172.7902,81.50,72,7,4.14,clear sky
3,3,Hobart,AU,-42.8794,147.3294,49.86,80,90,1.99,overcast clouds
4,4,Manjeshwar,IN,12.7194,74.8861,79.75,87,87,4.34,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input('What is the minimum temperature you would like for your trip? '))
max_temp=float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp']>=min_temp)&(city_data_df['Max Temp']<=max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carupano,VE,10.6678,-63.2585,77.83,89,96,8.93,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,73.45,68,75,9.22,broken clouds
2,2,Butaritari,KI,3.0707,172.7902,81.50,72,7,4.14,clear sky
4,4,Manjeshwar,IN,12.7194,74.8861,79.75,87,87,4.34,moderate rain
10,10,Bambous Virieux,MU,-20.3428,57.7575,75.45,73,40,12.66,light rain
...,...,...,...,...,...,...,...,...,...,...
660,660,Caravelas,BR,-17.7125,-39.2481,75.15,93,100,8.39,light rain
663,663,Madanapalle,IN,13.5500,78.5000,72.50,81,80,1.14,broken clouds
664,664,Saba,HN,15.4667,-86.2500,88.90,48,11,1.30,few clouds
665,665,Meulaboh,ID,4.1363,96.1285,76.05,92,99,3.29,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df[preferred_cities_df.isna().any(axis=1)]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
127,127,Grootfontein,NaN,-19.5667,18.1167,92.77,10,20,8.05,few clouds
218,218,Opuwo,NaN,-18.0607,13.8400,90.19,10,7,17.43,clear sky


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()

In [7]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carupano,VE,77.83,overcast clouds,10.6678,-63.2585,
1,Avarua,CK,73.45,broken clouds,-21.2078,-159.7750,
2,Butaritari,KI,81.50,clear sky,3.0707,172.7902,
4,Manjeshwar,IN,79.75,moderate rain,12.7194,74.8861,
10,Bambous Virieux,MU,75.45,light rain,-20.3428,57.7575,
12,San Patricio,US,84.02,clear sky,28.0170,-97.5169,
13,Chinsali,ZM,83.21,broken clouds,-10.5414,32.0816,
15,Dothan,US,76.17,overcast clouds,31.2232,-85.3905,
16,Sinnamary,GF,80.80,scattered clouds,5.3833,-52.9500,
17,Ribeira Grande,PT,70.20,scattered clouds,38.5167,-28.7000,


In [9]:
# 6a. Set parameters to search for hotels within 5000 meters.

#dictionary to drop the rows with no Hotel Name.
hotels_no_name=[]

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lng=row['Lng']
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6c. Get latitude and longitude from DataFrame
    hotel_df[["Lat", "Lng"]]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        pass
        print('IndexError',index)
        #indexes rows to be dropped with no Hotel Name
        if index not in hotels_no_name:
            hotels_no_name.append(index)

print(hotels_no_name)
hotel_df

IndexError 31
IndexError 216
IndexError 241
IndexError 284
IndexError 325
IndexError 346
IndexError 353
IndexError 359
IndexError 394
IndexError 417
IndexError 487
IndexError 538
IndexError 583
IndexError 586
IndexError 590
IndexError 641
[31, 216, 241, 284, 325, 346, 353, 359, 394, 417, 487, 538, 583, 586, 590, 641]


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carupano,VE,77.83,overcast clouds,10.6678,-63.2585,Hotel La dorada
1,Avarua,CK,73.45,broken clouds,-21.2078,-159.7750,Paradise Inn
2,Butaritari,KI,81.50,clear sky,3.0707,172.7902,Isles Sunset Lodge
4,Manjeshwar,IN,79.75,moderate rain,12.7194,74.8861,SOOFINAGAR
10,Bambous Virieux,MU,75.45,light rain,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
660,Caravelas,BR,75.15,light rain,-17.7125,-39.2481,Pousada dos Navegantes
663,Madanapalle,IN,72.50,broken clouds,13.5500,78.5000,Prithvi Lodge
664,Saba,HN,88.90,few clouds,15.4667,-86.2500,TORROJAMARU
665,Meulaboh,ID,76.05,light rain,4.1363,96.1285,Kost Pria


In [10]:
#Eventhough exists no Hotels Names, the DF shows no null rows.
hotel_df[hotel_df['Hotel Name'].isnull()]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotels_no_name, inplace=False)
clean_hotel_df=hotel_df
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carupano,VE,77.83,overcast clouds,10.6678,-63.2585,Hotel La dorada
1,Avarua,CK,73.45,broken clouds,-21.2078,-159.7750,Paradise Inn
2,Butaritari,KI,81.50,clear sky,3.0707,172.7902,Isles Sunset Lodge
4,Manjeshwar,IN,79.75,moderate rain,12.7194,74.8861,SOOFINAGAR
10,Bambous Virieux,MU,75.45,light rain,-20.3428,57.7575,Casa Tia Villa


In [12]:
# 8a. Create the output File (CSV)
output_data_file=('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and Max. Temp. of: {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(50.0, 31.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))